In [1]:
import copy
import operator
import copy
import numpy as np

In [15]:
def loadData():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    data = {}
    for line in simpDat:
        data[frozenset(line)] = 1
    return data
class node:
    def __init__(self, info, count = 1):
        self.info = info
        self.count = count
        self.children = []
        self.parent = []
class tree:
    def __init__(self, valueList, countList):
        self.root = node('Null')
        self.valueList = valueList
        self.countList = countList
        self.createTree()
    def createTree(self):
        for value in self.valueList:
            curNode = self.root
            while(len(curNode.children) != 0):
                curNode = curNode.children[0]
            curNode.children.append(node(value, self.countList[self.valueList.index(value)]))
            childNode = curNode.children[0]
            childNode.parent.append(curNode)

    def disp(self):
        curNode = self.root
        while(len(curNode.children)):
            print(curNode.info, curNode.count)
            curNode = curNode.children[0]
        print(curNode.info, curNode.count)
def FPtree(data, minSupport = 2):
    headers = {}
    for line in data:
        for item in line:
            headers[item] = headers.get(item, 0) + data[line]
    headerTable = {}
    for key in headers.keys():
        if headers[key] >= minSupport:
            headerTable[key] = headers[key]
    root = node('Null')
    for line in data:
        filterLine = {}
        for item in line:
            if item in headerTable.keys():
                filterLine[item] = headerTable[item]
        sortFilterLine = []
        for item, count in sorted(filterLine.items(), key = operator.itemgetter(1), reverse = True):
            sortFilterLine.append((item, list(line).count(item[0]) * data[line]))
        valueList = []
        countList = []
        for value, count in sortFilterLine:
            valueList.append(value)
            countList.append(count)
        newTree = tree(valueList, countList)
        root = mergeTree(root, newTree.root)
    return root, headerTable
def mergeTree(tree1, tree2):
    cur1 = tree1
    cur2 = tree2
    while(len(cur1.children) and len(cur2.children)):
        if cur1.info == cur2.info:
            cur1.count += cur2.count
            ind = NodeIndexOfTreeList(cur1.children, cur2.children[0])
            if ind == -1:
                cur1.children.append(cur2.children[0])
                return tree1
            else:
                cur1 = cur1.children[ind]
                cur2 = cur2.children[0]
    if len(cur1.children) == 0 and len(cur2.children) > 0:
        if cur1.info == cur2.info:
            cur1.count += cur2.count
            cur1.children.append(cur2.children[0])
    if len(cur2.children) == 0:
        if cur1.info == cur2.info:
            cur1.count += cur2.count
    tree1.count = 0
    return tree1
def NodeIndexOfTreeList(listNode, node):
    for i in range(len(listNode)):
        if node.info == listNode[i].info:
            return i
    return -1
def PrintTree(treeRoot):
    queue = [treeRoot]
    front = treeRoot
    curLast = treeRoot
    nextLast = treeRoot
    line = ''
    while(len(queue)):
        front = queue[0]
        queue = queue[1:]
        line += '{}:{}  '.format(front.info, front.count)
        if len(front.children) > 0:
            for item in front.children:
                queue.append(item)
            nextLast = front.children[-1]
        if curLast == front:
            print(line)
            line = ''
            curLast = nextLast
def getNode(root, info):
    nodeList = []
    queue = [root]
    front = root
    while(len(queue)):
        front = queue[0]
        queue = queue[1:]
        if front.info == info:
            nodeList.append(front)
        if len(front.children) > 0:
            for item in front.children:
                queue.append(item)
    return nodeList
def prefixPath(root, item):
    nodeList = getNode(root, item)
    path = {}
    for node in nodeList:
        items = []
        count = node.count
        while(node.parent):
            node = node.parent[0]
            if node.info == 'Null':
                break
            items.insert(0, node.info)
        if len(items) > 0:
            path[frozenset(items)] = count
    return path

In [16]:
def mineTree(tree, header, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(header.items(), key = operator.itemgetter(1), reverse = False)]
    for basePattern in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePattern)
        freqItemList.append(newFreqSet)
        condPatBases = prefixPath(tree, basePattern)
        condTree, h = FPtree(condPatBases, minSup)
        if len(h):
            mineTree(condTree, h, minSup, newFreqSet, freqItemList)
        print(newFreqSet)
        PrintTree(condTree)
        print('')

In [17]:
root, header = FPtree(loadData(), 3)
r = copy.deepcopy(root)
prefix = set([])
freqItemList = []
mineTree(root, header, 3, prefix, freqItemList)

{'r'}
Null:0  

{'y', 'x'}
Null:1  

{'x', 'y', 'z'}
Null:1  

{'y', 'z'}
Null:0  
x:3  

{'y'}
Null:0  
x:3  
z:3  

{'t', 'x'}
Null:1  

{'t', 'y', 'x'}
Null:1  

{'t', 'y'}
Null:0  
x:3  

{'t', 'y', 'z'}
Null:1  

{'x', 't', 'y', 'z'}
Null:1  

{'x', 't', 'z'}
Null:0  
y:3  

{'t', 'z'}
Null:0  
y:3  
x:3  

{'t'}
Null:0  
x:3  
y:3  
z:3  

{'x', 's'}
Null:1  

{'s'}
Null:0  
x:3  

{'z', 'x'}
Null:1  

{'x'}
Null:0  
z:3  

{'z'}
Null:1  



In [18]:
freqItemList

[{'r'},
 {'y'},
 {'x', 'y'},
 {'y', 'z'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'x'},
 {'t', 'y'},
 {'t', 'x', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'x'},
 {'x', 'z'},
 {'z'}]

In [19]:
data = {}
for line in [line.split() for line in open('kosarak.dat').readlines()]:
    data[frozenset(line)] = 1

In [20]:
root, header = FPtree(data, 100000)

In [22]:
freqList = []
mineTree(root, header, 100000, set([]), freqList)

{'1', '6'}
Null:1  

{'1'}
Null:0  
6:107404  

{'11', '3'}
Null:0  

{'6', '3'}
Null:1  

{'3'}
Null:0  
6:186289  11:0  
11:0  

{'11'}
Null:0  

{'6'}
Null:1  



In [25]:
freqList

[{'1'}, {'1', '6'}, {'3'}, {'11', '3'}, {'3', '6'}, {'11'}, {'6'}]